In [ ]:
import json, re,  os, time, random 
from bs4 import BeautifulSoup  
import pymongo , bson
import nltk
from nltk.corpus import stopwords
sw=stopwords.words('english')
import pandas as pd 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from datetime import datetime

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn) 
db = client.dhwy
collection = db.posts 

In [ ]:
options = Options()
options.add_argument("--start-minimized")  

In [ ]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install() ,chrome_options=options) 
base = 'https://dallashwy.com'
driver.get(f"{base}/index.php?forums/politics.8/")   



In [ ]:
# BeautifulSoup(driver.page_source, 'html.parser')
base = 'https://dallashwy.com'
driver.get(f"{base}/index.php?forums/politics.8/")  
pages = 0
while True:  
    soup = BeautifulSoup(driver.page_source, 'html.parser')  
    try:
        get_topics( driver.current_url  )
    except e:
        print(e)
            
    nxt = soup.select('link[rel="next"]')  
    if len(nxt) > 0 and pages <= 250: 
        driver.get(f"{base}{nxt[0]['href']}")  
        pages = pages + 1
    else: 
        break

In [ ]:
def get_topics(url):
    driver.get(f"{url}")
    topics = driver.find_elements_by_css_selector('.structItem--thread .structItem-title a') 
    hrefs = []
    for topic in topics[:20]:  
        hrefs.append(topic.get_attribute('href'))
    for href in hrefs:  
        driver.get(f"{href}") 
        while True:   
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            print_thread(driver)
            nxt = soup.select('link[rel="next"]')  
            if len(nxt) > 0:   
                driver.get(f"{base}{nxt[0]['href']}") 
                try:
                    print_thread(driver)
                except e:
                    print(e)
            else: 
                break 

In [ ]:
def print_thread(driver): 
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    arts = soup.select('article.message')
    i = 0
    d={}
    for a in arts:  
        d['key']=a['id']
        d['author']=a['data-author']
        d['title']=soup.select('title')[0].get_text() 
        d['url']=base+a.select('header a')[0]['href']
        ele = a.select('time.u-dt') 
        if len(ele) > 0: 
            d['date']=ele[0].get_text() 
        ele = a.select('.message-body .bbWrapper') 
        if len(ele) > 0:  
            d['mess']=ele[0].get_text()
        ele = a.select('a.link--external') 
        if len(ele) > 0:  
            d['mess']=d['mess']+ele[0]['href']
        ele = a.select('.bbImageWrapper') 
        if len(ele) > 0:  
            d['mess']=d['mess']+ele[0]['data-src']
        # print(d)
        #collection.delete_one( d )
        #collection.insert_one( d ) 
        collection.update( { 'key':  d['key']  }, d, True   ) 

In [ ]:
df = pd.DataFrame(list(collection.find()))
df[:2]

In [ ]:
dff = df.loc[df.author.str.contains('Far West')] 
#dff = df.loc[df.mess.str.contains('vacc')] 
for i,r, in dff.iterrows():
    print( r['author'] + ' ' +  r['date']+ ' ' +  r['title'])
    print(r['mess'] )
    print(r['url'] )

In [ ]:
pages = driver.find_elements_by_css_selector('.pageNavWrapper')
for p in pages:
    print(p)

In [ ]:
for e in eles:
    ele=e.find_element_by_css_selector('.structItem-title a')
    print(ele.get_attribute('id'))

In [ ]:
eles?

In [ ]:
elms = driver.find_elements(By.XPATH, '//ul[contains(@id, "search-results")]//h3/a') 
hrefs = [e.get_attribute('href') for e in elms]  

In [ ]:

for url in hrefs[:12]:
    driver.execute_script( 'window.open();')   
    driver.switch_to_window(driver.window_handles[ len(driver.window_handles)-1])
    driver.get(url)
driver.window_handles[0]